In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.ensemble import GradientBoostingRegressor
from collections import defaultdict


/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
from importlib import reload
from data_cleaning import data_cleaning as dc
from models import contract_regressor as cr

In [178]:
reload(dc)

<module 'data_cleaning.data_cleaning' from '/Users/colinshanahan/galvanize/nhl-contract-predictions/data_cleaning/data_cleaning.py'>

In [67]:
def clean_FA():
    df = pd.read_csv('../data/free_agents_2019.csv')
    df['signing_status'] = df['ufa_status'].apply(lambda x: 1 if 'UFA' else 0)
    df = df[(df.position != 'G') & (df.position != 27) & (df.position != 30)]
    df['forward'] = (df.position != 'D').replace([True, False], [1, 0])
    df['Season_Player'] = '2018 ' + df['Player']
    df.set_index(df['Season_Player'], inplace=True)
    df.drop('Season_Player', axis=1, inplace=True)
    fa = df[['Player', 'age', 'forward', 'signing_status']]
    fa['signing_year'] = 2019
    return fa

In [102]:
df = dc.clean_FA()
df.sample(5)

,Player,signing_age,forward,signing_status,signing_year
Season_Player,,,,,
2018 Dysin Mayo,Dysin Mayo,22,0,1,2019
2018 Alex Gallant,Alex Gallant,26,1,1,2019
2018 Julius Honka,Julius Honka,23,0,1,2019
2018 Chase Lang,Chase Lang,22,1,1,2019
2018 Michael Downing,Michael Downing,23,0,1,2019


In [103]:
df.shape

(540, 5)

In [105]:
len(df.Player.unique())

539

In [126]:
st = dc.clean_features_data(sql=False, new_fas=True)

In [116]:
st.head(6)

,Player,signing_age,forward,signing_status,signing_year,GP,TOI,Goals,Total Assists,First Assists,...,mean Shots/60,Hits/60,mean Hits/60,PIM/60,mean PIM/60,Penalties Drawn/60,mean Penalties Drawn/60,mean Faceoffs pct,Goalness,mean Goalness
Season_Player,,,,,,,,,,,,,,,,,,,,,
2018 Mitchell Marner,Mitchell Marner,21,1,1,2019,82.0,1625.433333,26.0,68.0,52.0,...,8.485720,1.070484,1.421323,0.812091,1.210235,0.738265,0.830278,0.986111,0.273684,0.297778
2018 Brayden Point,Brayden Point,23,1,1,2019,79.0,1494.600000,41.0,51.0,28.0,...,7.464184,1.244480,1.349457,1.124047,0.927752,0.923324,0.815297,0.923423,0.440860,0.457286
2018 Mikko Rantanen,Mikko Rantanen,22,1,1,2019,74.0,1542.300000,31.0,56.0,34.0,...,6.822104,2.295273,1.624311,2.100759,1.488951,1.050379,0.974586,0.990654,0.352273,0.380952
2018 Artemi Panarin,Artemi Panarin,27,1,1,2019,79.0,1568.666667,28.0,59.0,44.0,...,7.881760,0.535487,0.650558,0.879728,0.850730,0.841479,0.850730,0.166667,0.318182,0.352459
2018 Sebastian Aho,Sebastian Aho,21,1,1,2019,82.0,1652.366667,30.0,53.0,29.0,...,8.223319,2.360251,2.465235,0.944100,0.986094,0.980412,0.898050,0.928571,0.357143,0.392157
2018 Sebastian Aho,Sebastian Aho,21,1,1,2019,82.0,1652.366667,30.0,53.0,29.0,...,8.223319,2.360251,2.465235,0.944100,0.986094,0.980412,0.898050,0.928571,0.357143,0.392157


In [100]:
xcols = ['forward', 'signing_age', 'signing_status'] + list(st.columns)[-198:]
xcols

['forward',
 'signing_age',
 'signing_status',
 'GP',
 'TOI',
 'Goals',
 'Total Assists',
 'First Assists',
 'Second Assists',
 'Total Points',
 'Shots',
 'iCF',
 'iFF',
 'iSCF',
 'iHDCF',
 'Rush Attempts',
 'Rebounds Created',
 'PIM',
 'Total Penalties',
 'Minor',
 'Major',
 'Misconduct',
 'Penalties Drawn',
 'Giveaways',
 'Takeaways',
 'Hits',
 'Hits Taken',
 'Shots Blocked',
 'Faceoffs Won',
 'Faceoffs Lost',
 'sum GP',
 'sum TOI',
 'sum Goals',
 'sum Total Assists',
 'sum First Assists',
 'sum Second Assists',
 'sum Total Points',
 'sum Shots',
 'sum iCF',
 'sum iFF',
 'sum iSCF',
 'sum iHDCF',
 'sum Rush Attempts',
 'sum Rebounds Created',
 'sum PIM',
 'sum Total Penalties',
 'sum Minor',
 'sum Major',
 'sum Misconduct',
 'sum Penalties Drawn',
 'sum Giveaways',
 'sum Takeaways',
 'sum Hits',
 'sum Hits Taken',
 'sum Shots Blocked',
 'sum Faceoffs Won',
 'sum Faceoffs Lost',
 'mean GP',
 'mean TOI',
 'mean Goals',
 'mean Total Assists',
 'mean First Assists',
 'mean Second Assists

In [94]:
list(st.columns)

['Player',
 'signing_age',
 'forward',
 'signing_status',
 'signing_year',
 'GP',
 'TOI',
 'Goals',
 'Total Assists',
 'First Assists',
 'Second Assists',
 'Total Points',
 'Shots',
 'iCF',
 'iFF',
 'iSCF',
 'iHDCF',
 'Rush Attempts',
 'Rebounds Created',
 'PIM',
 'Total Penalties',
 'Minor',
 'Major',
 'Misconduct',
 'Penalties Drawn',
 'Giveaways',
 'Takeaways',
 'Hits',
 'Hits Taken',
 'Shots Blocked',
 'Faceoffs Won',
 'Faceoffs Lost',
 'sum GP',
 'sum TOI',
 'sum Goals',
 'sum Total Assists',
 'sum First Assists',
 'sum Second Assists',
 'sum Total Points',
 'sum Shots',
 'sum iCF',
 'sum iFF',
 'sum iSCF',
 'sum iHDCF',
 'sum Rush Attempts',
 'sum Rebounds Created',
 'sum PIM',
 'sum Total Penalties',
 'sum Minor',
 'sum Major',
 'sum Misconduct',
 'sum Penalties Drawn',
 'sum Giveaways',
 'sum Takeaways',
 'sum Hits',
 'sum Hits Taken',
 'sum Shots Blocked',
 'sum Faceoffs Won',
 'sum Faceoffs Lost',
 'mean GP',
 'mean TOI',
 'mean Goals',
 'mean Total Assists',
 'mean First Ass

In [123]:
st.shape

(246, 203)

In [124]:
st.drop_duplicates(subset = ['Player', 'signing_age'], inplace=True)
st.shape

(245, 203)

In [113]:
len(st.Player.unique())

245

In [179]:
modst = dc.clean_features_data(sql=False, new_fas=False)

In [180]:
modst.shape

(1753, 224)

In [144]:
df = pd.read_csv('../data/up_all_pst.csv')
dfsummable = df.drop(['SH%', 'Faceoffs %', 'IPP'], axis=1)

summable_stats = ['GP', 'TOI', 'Goals',
            'Total Assists', 'First Assists', 'Second Assists', 'Total Points',
           'Shots', 'iCF', 'iFF', 'iSCF', 'iHDCF', 'Rush Attempts',
           'Rebounds Created', 'PIM', 'Total Penalties', 'Minor', 'Major',
           'Misconduct', 'Penalties Drawn', 'Giveaways', 'Takeaways', 'Hits',
           'Hits Taken', 'Shots Blocked', 'Faceoffs Won', 'Faceoffs Lost']


#read in on_ice relative stats per season and over 3 year window
oirel = pd.read_csv('../data/up_all_oirel.csv')
woirel = pd.read_csv('../data/up_all_woirel.csv')


#fix column labeling
oirel['Season'] = oirel['Unnamed: 0.1']
oirel.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
woirel['Season'] = woirel['Unnamed: 0.1']
woirel.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)


#set indices to be season/player combo
oirel['Season_Player'] = oirel.Season.apply(str)
oirel.Season_Player = oirel.Season_Player + ' ' + oirel.Player
woirel['Season_Player'] = woirel.Season.apply(str)
woirel.Season_Player = woirel.Season_Player + ' ' + woirel.Player
oirel.set_index(oirel.Season_Player, inplace=True)
woirel.set_index(woirel.Season_Player, inplace=True)

In [145]:
contracts = dc.clean_contracts_data()

In [146]:
contracts.shape

(1762, 27)

In [149]:
len(contracts.Player.unique())

935

In [150]:
len(contracts.contract_id.unique())

1762

In [147]:
#set up default dictionaries to hold individual player stats
allstats = defaultdict(pd.DataFrame)
allsumstats = defaultdict(pd.DataFrame)
allmeanstats = defaultdict(pd.DataFrame)

In [148]:
#Give each player it's own data frame of stats all linked together in a dictionary with
#player names as keys
#Take season total stats and aggregate them over a 3 year window
for player in contracts['Player'].unique():
    allstats[player] = dfsummable[df.Player == player]
    allstats[player].sort_values(by='Season', inplace=True)
    allstats[player]['Season_index'] = pd.date_range(end='2019',
                                               periods = allstats[player].shape[0],
                                               freq='Y')
    allstats[player].set_index(allstats[player].Season_index, inplace=True)
    allsumstats[player] = (allstats[player][summable_stats].rolling(window=3, min_periods=1)
                  .agg(np.sum))
    allmeanstats[player] = (allstats[player][summable_stats].rolling(window=3, min_periods=1)
              .agg(np.mean))
    allmeanstats[player]['SH%'] = allsumstats[player]['Shots'] / allsumstats[player]['Goals']
    allmeanstats[player]['Faceoffs %'] = allsumstats[player]['Faceoffs Won'] / (
    allsumstats[player]['Faceoffs Won'] + allsumstats[player]['Faceoffs Lost'])

In [151]:
scols, mcols = [], []
#Relabeling sum and mean columns for clarity, arbitrarily using Pavelski for convenience,
#could be any player
for i in allsumstats['Joe Pavelski'].columns:
    scols.append('sum '+i)
for i in allmeanstats['Joe Pavelski'].columns:
    mcols.append('mean '+i)
for p in allstats:
    allsumstats[p].columns = scols
    allmeanstats[p].columns = mcols
    allstats[p] = pd.concat([allstats[p], allsumstats[p], allmeanstats[p]], axis=1)

In [152]:
len(allstats)

935

In [158]:
allsumstats['Tomas Hertl'].shape, allmeanstats['Tomas Hertl'].shape

((6, 27), (6, 29))

In [153]:
#get a combined dataframe with all relevant player/contract years
players = list(allstats.keys())
allallstats = pd.DataFrame()

for p in players:
    allallstats = pd.concat([allallstats, allstats[p].set_index(allstats[p].Season_Player)])


In [154]:
allallstats.shape

(5673, 89)

In [159]:
#drop duplicate column names
m_oirel = oirel.drop(['Player', 'Team', 'Position', 'GP', 'TOI', 'Season'], axis=1)
m_woirel = woirel.drop(['Player', 'Team', 'Position', 'GP', 'TOI', 'Season'], axis=1)

In [160]:
#relabel 3yr window columns for clarity
wcols = []
for i in m_woirel.columns:
    wcols.append('3yr ' + i)
m_woirel.columns = wcols

In [161]:
#merge on ice relative stats to the other stats
allallstats = pd.merge(allallstats, m_oirel, on = 'Season_Player')
allallstats = pd.merge(allallstats, m_woirel, on = 'Season_Player')

In [162]:
allallstats.shape

(5307, 182)

In [167]:
#remove contracts signed before 2010 (lack of stats)
table = contracts[contracts.signing_year > 2009]#.set_index('contract_id')


In [168]:
table.shape

(1753, 27)

In [169]:
#add column to line up contract years and stats years
table['year_match'] = table.signing_year - 1

#merge contracts and stats into a single table
table = pd.merge(table, allallstats,
            how = 'left', left_on = ['Player', 'year_match'],
            right_on = ['Player', 'Season'])
#drop rows that had missing seasons / indexing issues leading to NaNs
#1700 rows down to 1440 ---- 
#table.dropna(inplace=True)

In [175]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 564 entries, 4 to 1703
Columns: 209 entries, player_id to 3yr Season_Player
dtypes: bool(1), datetime64[ns](2), float64(148), int64(12), object(46)
memory usage: 921.5+ KB


In [500]:
modst = dc.clean_features_data(sql=False, new_fas=False)

In [661]:
modst.to_csv('../data/clean_stats_final.csv')

In [594]:
reload(cr)
reload(dc)

<module 'data_cleaning.data_cleaning' from '/Users/colinshanahan/galvanize/nhl-contract-predictions/data_cleaning/data_cleaning.py'>

In [595]:
con_reg = cr.ContractRegressor()

In [596]:
X = con_reg.pick_cols(modst)
y = modst[['cap_pct', 'length']]
X.shape, y.shape

((1191, 197), (1191, 2))

In [597]:
X, y = con_reg.drop_duplicate_rows(X), con_reg.drop_duplicate_rows(y)

In [598]:
X = con_reg.impute(X)

In [643]:
split = (
    train_test_split(X, y['cap_pct'], y['length'], 
                     test_size = 0.15))

X_train, X_test, yp_train, yp_test, yl_train, yl_test = split

In [680]:
train = X_train.copy()
train['cap_pct'] = yp_train
train['length'] = yl_train

In [682]:
test = X_test.copy()
test['cap_pct'] = yp_test
test['length'] = yl_test

In [683]:
train.to_csv('../data/train.csv')
test.to_csv('../data/test.csv')

In [644]:
con_reg.fit(X_train, yp_train, yl_train)

In [645]:
con_reg.predict_global_mean(yp_train, yp_test, yl_train, yl_test)
print()
preds = con_reg.predict(X_test)
con_reg.score(X_test, yp_test, yl_test)

RMSE pick mean cap_pct: 2.9%
    translates to 2019 Cap Hit of: $2407000.0
RMSE pick mean length: 1.9 years

RMSE Cap_pct: 0.97%
    translates to 2019 Cap Hit of: $805100.0
RMSE Length: 1.0 years


(0.97, 805100.0, 1.0)

In [690]:
tot_val = yp_test*yl_test * 83000000 // 100
yp_hat = con_reg.sal_model.predict(X_test)
Xl_test = X_test.copy()
Xl_test['pred_cap_pct'] = yp_hat
yl_hat = con_reg.len_model.predict(Xl_test)
tot_val_hat = yp_hat*yl_hat * 83000000 // 100

tval_mean_rmse = con_reg.rmse((2.90*1.9*83000000//100), tot_val)

tval_pred_rmse = con_reg.rmse(tot_val_hat, tot_val)

In [692]:
t_val_improvement = (tval_mean_rmse-tval_pred_rmse)/tval_mean_rmse

In [694]:
print('RMSE for predicting mean Total Value: ${}'.format(round(tval_mean_rmse,-3)))
print('RMSE for Gradient Boosted Model Total Value: ${}'.format(round(tval_pred_rmse, -3)))
print('Improvement: {}%'.format(round(t_val_improvement*100, 1)))

RMSE for predicting mean Total Value: $17899000.0
RMSE for Gradient Boosted Model Total Value: $7523000.0
Improvement: 58.0%


In [508]:
fas = dc.clean_features_data(sql=False, new_fas=True)

In [660]:
fas.to_csv('../data/free_agent_cleaned.csv')

In [532]:
fas.shape

(352, 202)

In [512]:
fas.head(10)

,Player,signing_age,forward,signing_status,signing_year,GP,TOI,Goals,Total Assists,First Assists,...,mean Shots/60,Hits/60,mean Hits/60,PIM/60,mean PIM/60,Penalties Drawn/60,mean Penalties Drawn/60,mean Faceoffs pct,Goalness,mean Goalness
Season_Player,,,,,,,,,,,,,,,,,,,,,
2018 Mitchell Marner,Mitchell Marner,21,1,1,2019,82.0,1625.433333,26.0,68.0,52.0,...,8.485720,1.070484,1.421323,0.812091,1.210235,0.738265,0.830278,0.496503,0.273684,0.297778
2018 Brayden Point,Brayden Point,23,1,1,2019,79.0,1494.600000,41.0,51.0,28.0,...,7.464184,1.244480,1.349457,1.124047,0.927752,0.923324,0.815297,0.480094,0.440860,0.457286
2018 Mikko Rantanen,Mikko Rantanen,22,1,1,2019,74.0,1542.300000,31.0,56.0,34.0,...,6.822104,2.295273,1.624311,2.100759,1.488951,1.050379,0.974586,0.497653,0.352273,0.380952
2018 Artemi Panarin,Artemi Panarin,27,1,1,2019,79.0,1568.666667,28.0,59.0,44.0,...,7.881760,0.535487,0.650558,0.879728,0.850730,0.841479,0.850730,0.142857,0.318182,0.352459
2018 Sebastian Aho,Sebastian Aho,21,1,1,2019,82.0,1652.366667,30.0,53.0,29.0,...,8.223319,2.360251,2.465235,0.944100,0.986094,0.980412,0.898050,0.481481,0.357143,0.392157
2018 Sebastian Aho,Sebastian Aho,21,1,1,2019,82.0,1652.366667,30.0,53.0,29.0,...,8.223319,2.360251,2.465235,0.944100,0.986094,0.980412,0.898050,0.481481,0.357143,0.392157
2018 Matthew Tkachuk,Matthew Tkachuk,21,1,1,2019,80.0,1407.916667,34.0,43.0,26.0,...,8.719420,4.432080,4.043083,2.556970,3.637151,1.363717,2.045897,0.396226,0.435897,0.405714
2018 Matt Duchene,Matt Duchene,28,1,1,2019,73.0,1374.900000,31.0,39.0,25.0,...,7.304871,1.221907,1.819221,0.349116,0.531772,0.654593,0.699700,0.566140,0.436620,0.444444
2018 Timo Meier,Timo Meier,22,1,1,2019,78.0,1323.416667,30.0,36.0,20.0,...,11.077986,4.488382,5.447524,2.493546,2.357883,1.632139,1.565147,0.444444,0.447761,0.495413


In [646]:
faX = con_reg.pick_cols(fas)
#fay = fa[['cap_pct', 'length']]
faX.shape

(352, 197)

In [647]:
faX = con_reg.drop_duplicate_rows(faX)
faX = con_reg.impute(faX)
faX.shape

(351, 197)

In [648]:
faX.head(10)

,forward,signing_age,signing_status,GP,TOI,Goals,Total Assists,First Assists,Second Assists,Total Points,...,Shots/60,mean Shots/60,Hits/60,mean Hits/60,PIM/60,mean PIM/60,Penalties Drawn/60,mean Penalties Drawn/60,Goalness,mean Goalness
Season_Player,,,,,,,,,,,,,,,,,,,,,
2018 Mitchell Marner,1.0,21.0,1.0,82.0,1625.433333,26.0,68.0,52.0,16.0,94.0,...,8.600783,8.485720,1.070484,1.421323,0.812091,1.210235,0.738265,0.830278,0.273684,0.297778
2018 Brayden Point,1.0,23.0,1.0,79.0,1494.600000,41.0,51.0,28.0,23.0,92.0,...,7.707748,7.464184,1.244480,1.349457,1.124047,0.927752,0.923324,0.815297,0.440860,0.457286
2018 Mikko Rantanen,1.0,22.0,1.0,74.0,1542.300000,31.0,56.0,34.0,22.0,87.0,...,7.508267,6.822104,2.295273,1.624311,2.100759,1.488951,1.050379,0.974586,0.352273,0.380952
2018 Artemi Panarin,1.0,27.0,1.0,79.0,1568.666667,28.0,59.0,44.0,15.0,87.0,...,7.305567,7.881760,0.535487,0.650558,0.879728,0.850730,0.841479,0.850730,0.318182,0.352459
2018 Sebastian Aho,1.0,21.0,1.0,82.0,1652.366667,30.0,53.0,29.0,24.0,83.0,...,8.823707,8.223319,2.360251,2.465235,0.944100,0.986094,0.980412,0.898050,0.357143,0.392157
2018 Matthew Tkachuk,1.0,21.0,1.0,80.0,1407.916667,34.0,43.0,26.0,17.0,77.0,...,8.821545,8.719420,4.432080,4.043083,2.556970,3.637151,1.363717,2.045897,0.435897,0.405714
2018 Matt Duchene,1.0,28.0,1.0,73.0,1374.900000,31.0,39.0,25.0,14.0,70.0,...,7.506000,7.304871,1.221907,1.819221,0.349116,0.531772,0.654593,0.699700,0.436620,0.444444
2018 Timo Meier,1.0,22.0,1.0,78.0,1323.416667,30.0,36.0,20.0,16.0,66.0,...,11.334299,11.077986,4.488382,5.447524,2.493546,2.357883,1.632139,1.565147,0.447761,0.495413
2018 Joe Pavelski,1.0,34.0,1.0,75.0,1428.300000,38.0,26.0,17.0,9.0,64.0,...,7.897501,8.494759,2.898551,3.845690,0.924176,1.277506,1.050200,0.961422,0.584615,0.447236


In [674]:
import pickle

In [675]:
pickle.dump(con_reg, open('../models/final_model.p', 'wb'))

In [649]:
fa_preds = con_reg.predict(faX)

In [650]:
fa_preds_df = pd.DataFrame(index=faX.index)

In [651]:
fa_preds_df['cap_pct'] = fa_preds[0]
fa_preds_df['length'] = fa_preds[1]

In [676]:
fa_preds_df.describe()

,cap_pct,length
count,351.000000,351.000000
mean,2.192678,1.980627
std,2.210811,1.299372
min,0.520000,0.700000
25%,0.920000,1.300000
50%,1.070000,1.500000
75%,2.505000,2.000000
max,11.190000,7.900000


In [653]:
fa_preds_df['2019_cap_hit'] = fa_preds_df.cap_pct * 83000000 // 100

In [654]:
fa_preds_df['2019_cap_hit'] = fa_preds_df['2019_cap_hit'].apply(lambda x: int(round(x, -3)))

In [655]:
def int_to_mon_str(x):
    s = str(x)
    if x < 1000000:
        s = '$' + s[:3] + ',' + s[3:]
    elif x < 10000000:
        s = '$' + s[0] + ',' + s[1:4] + ',' + s[4:]
    else:
        s = '$' + s[0:2] + ',' + s[2:5] + ',' + s[5:]
    return s

In [656]:
fa_preds_df['2019_cap_hit'] = fa_preds_df['2019_cap_hit'].apply(int_to_mon_str)

In [657]:
fa_preds_df['cap_pct'] = fa_preds_df['cap_pct'].apply(lambda x: round(x, 2))
fa_preds_df['length'] = fa_preds_df['length'].apply(lambda x: round(x, 1))

In [658]:
fa_preds_df

,cap_pct,length,2019_cap_hit
Season_Player,,,
2018 Mitchell Marner,10.99,7.6,"$9,121,000"
2018 Brayden Point,10.53,7.9,"$8,741,000"
2018 Mikko Rantanen,11.10,6.0,"$9,216,000"
2018 Artemi Panarin,11.19,6.1,"$9,288,000"
2018 Sebastian Aho,9.71,7.1,"$8,059,000"
2018 Matthew Tkachuk,8.87,7.0,"$7,360,000"
2018 Matt Duchene,8.46,6.0,"$7,020,000"
2018 Timo Meier,6.47,5.7,"$5,372,000"
2018 Joe Pavelski,9.32,4.6,"$7,733,000"


In [659]:
fa_preds_df.to_csv('../data/free_agent_preds_2019.csv')

In [590]:
x = np.array([1.342,2.3453,3.2352,4.25434,5.343])

In [586]:
x2 = map(round, x)

In [591]:
x3 = np.array(list(map(round, x)))

In [592]:
x3.shape

(5,)

In [593]:
x3

array([1., 2., 3., 4., 5.])

In [672]:
def prediction_cleaning(sal_preds, len_preds, idx, year = 2019):
    '''
    Output cap hit and length predictions into a readable pandas dataframe.
    Inputs:
    sal_preds: 1-D numpy array
    len_preds: 1-D numpy array
    idx: the index of the original data's DataFrame
    Returns:
    A 3 column pandas dataframe
    '''
    #create dataframe
    preds_df = pd.DataFrame(index=idx)
    preds_df['cap_pct'] = sal_preds
    preds_df['length'] = len_preds

    #A helper function that improves the readability of our output for cap_hit in dollars
    def int_to_mon_str(x):
        s = str(x)
        if x < 1000000:
            s = '$' + s[:3] + ',' + s[3:]
        elif x < 10000000:
            s = '$' + s[0] + ',' + s[1:4] + ',' + s[4:]
        else:
            s = '$' + s[0:2] + ',' + s[2:5] + ',' + s[5:]
        return s

    #add column for cap_hit in dollars
    print(type(preds_df['cap_pct'][0]))
    preds_df['2019_cap_hit'] = (fa_preds_df.cap_pct * 83000000 // 100).apply(int)
    print(type(preds_df['2019_cap_hit'][0]))
    preds_df['2019_cap_hit'] = fa_preds_df['2019_cap_hit'].apply(lambda x: round(x, -3))
    preds_df['2019_cap_hit'] = fa_preds_df['2019_cap_hit'].apply(int_to_mon_str)

    #Improve readability of prediction columns
    preds_df['cap_pct'] = fa_preds_df['cap_pct'].apply(lambda x: round(x, 2))
    preds_df['length'] = fa_preds_df['length'].apply(lambda x: round(x, 1))

    return preds_df

In [673]:
prediction_cleaning(fa_preds[0], fa_preds[1], faX.index)

<class 'numpy.float64'>
<class 'numpy.int64'>


TypeError: type str doesn't define __round__ method

In [684]:
(2.9 - 0.97)/2.9

0.6655172413793103

In [685]:
(1.9-1)/1.9

0.47368421052631576